# Split PharmacotherapyDB into multiple pieces

Tong Shu Li

For cross validation, the original training data needs to be split into multiple pieces in order to keep training and testing data separate.

In [1]:
import pandas as pd
import numpy as np

from itertools import product

In [2]:
np.__version__

'1.13.1'

In [3]:
np.random.seed(20161018)

---

## Load PharmacotherapyDB

In [4]:
goldstd = pd.read_csv("data/indications.tsv", sep = '\t')

In [5]:
goldstd.shape

(1388, 7)

In [6]:
goldstd.head()

,doid_id,drugbank_id,disease,drug,category,n_curators,n_resources
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,2,1
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,1,4
2,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM,1,3
3,DOID:10652,DB00989,Alzheimer's disease,Rivastigmine,DM,1,3
4,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM,3,1


In [7]:
goldstd["category"].value_counts()

DM     755
SYM    390
NOT    243
Name: category, dtype: int64

## Remove useless columns

The `n_curators` and `n_resources` columns are not needed for the subsequent portions of the pipeline. The columns names need to also be standardized for simplicity and expandability.

In [8]:
goldstd = (goldstd
    .drop(["n_curators", "n_resources"], axis=1)
    .rename(columns={
        "drugbank_id": "chemical_id",
        "drug": "chemical_name",
        "doid_id": "disease_id",
        "disease": "disease_name"
    })
)

In [9]:
goldstd.head()

,disease_id,chemical_id,disease_name,chemical_name,category
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM
2,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM
3,DOID:10652,DB00989,Alzheimer's disease,Rivastigmine,DM
4,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM


---

## Number of unique chemicals and diseases

In [10]:
goldstd["disease_id"].nunique()

97

In [11]:
goldstd["chemical_id"].nunique()

601

In [12]:
# number of unique chemical/disease combinations

goldstd["disease_id"].nunique() * goldstd["chemical_id"].nunique()

58297

## Split into multiple pieces

For K-fold validation, the entire workflow needs to be run K times. The value of K is chosen to be 5 to avoid excessive computational requirements.

We will split the data by assigning each piece of data a number from 0 to K-1, and group data rows according to the piece number. This will ensure that each row of data is used, and that the ratios of true/false examples per group is the same.

In [13]:
K = 5
goldstd["piece"] = np.random.randint(0, K, len(goldstd))

In [14]:
goldstd.head()

,disease_id,chemical_id,disease_name,chemical_name,category,piece
0,DOID:10652,DB00843,Alzheimer's disease,Donepezil,DM,0
1,DOID:10652,DB00674,Alzheimer's disease,Galantamine,DM,0
2,DOID:10652,DB01043,Alzheimer's disease,Memantine,DM,2
3,DOID:10652,DB00989,Alzheimer's disease,Rivastigmine,DM,4
4,DOID:10652,DB00245,Alzheimer's disease,Benzatropine,SYM,1


In [15]:
goldstd["piece"].value_counts()

0    291
1    289
2    275
4    269
3    264
Name: piece, dtype: int64

In [16]:
goldstd["piece"].value_counts(normalize = True)

0    0.209654
1    0.208213
2    0.198127
4    0.193804
3    0.190202
Name: piece, dtype: float64

Each piece has almost exactly 20% of the data.

In [17]:
# verify that the true/false ratios for each piece is similar
for group, df in goldstd.groupby("piece"):
    print("Group:", group)
    print(df["category"].value_counts())
    print()

Group: 0
DM     163
SYM     79
NOT     49
Name: category, dtype: int64

Group: 1
DM     160
SYM     81
NOT     48
Name: category, dtype: int64

Group: 2
DM     158
SYM     72
NOT     45
Name: category, dtype: int64

Group: 3
DM     134
SYM     76
NOT     54
Name: category, dtype: int64

Group: 4
DM     140
SYM     82
NOT     47
Name: category, dtype: int64



---

## Data separation

Ensure that all possible chemical-disease pairs in the holdout set are missing from the training data. This is to ensure that the algorithm never sees data which will be used to test the trained model.

In [18]:
def all_pairs(df):
    chem = df["chemical_id"].unique()
    dise = df["disease_id"].unique()
    
    return set(product(chem, dise))

def pair_to_df(pairs):
    return pd.DataFrame(list(pairs), columns = ["chemical_id", "disease_id"])

def df_to_pairs(df):
    return set(zip(df["chemical_id"], df["disease_id"]))

In [19]:
def split_data(withheld):
    holdout = goldstd.query("piece == @withheld")
    train = goldstd.query("piece != @withheld")
    
    # create all chemical-disease pairs
    holdout_p = all_pairs(holdout)
    train_p = all_pairs(train)
    
    train_assumed_false = train_p - df_to_pairs(train)
    
    overlap = holdout_p & train_assumed_false
    
    print("Number of overlaps for holdout set {}: {}".format(withheld, len(overlap)))

    train_assumed_false -= overlap
    
    # check that no negative training examples are in the holdout set
    assert train_assumed_false.isdisjoint(holdout_p)
    
    # check that there is no pharmacotherapydb overlap
    assert df_to_pairs(train).isdisjoint(df_to_pairs(holdout))
    
    # there will be some training examples which are potential candidates of the holdout
    
    train_df = train.append(pair_to_df(train_assumed_false))
    
    holdout_df = pair_to_df(holdout_p).merge(holdout, how = "left", on = ["chemical_id", "disease_id"])
    
    return (holdout_df, train_df)

In [20]:
holdouts = dict()
training = dict()

for withheld in range(K):
    hold, train = split_data(withheld)
    
    holdouts[withheld] = hold.sort_values(["chemical_id", "disease_id"])
    training[withheld] = train.sort_values(["chemical_id", "disease_id"])
    
    print(holdouts[withheld]["category"].value_counts())
    print()
    print(training[withheld]["category"].value_counts())
    print("-----")

Number of overlaps for holdout set 0: 10122
DM     163
SYM     79
NOT     49
Name: category, dtype: int64

DM     592
SYM    311
NOT    194
Name: category, dtype: int64
-----
Number of overlaps for holdout set 1: 8972
DM     160
SYM     81
NOT     48
Name: category, dtype: int64

DM     595
SYM    309
NOT    195
Name: category, dtype: int64
-----
Number of overlaps for holdout set 2: 8511
DM     158
SYM     72
NOT     45
Name: category, dtype: int64

DM     597
SYM    318
NOT    198
Name: category, dtype: int64
-----
Number of overlaps for holdout set 3: 8967
DM     134
SYM     76
NOT     54
Name: category, dtype: int64

DM     621
SYM    314
NOT    189
Name: category, dtype: int64
-----
Number of overlaps for holdout set 4: 9006
DM     140
SYM     82
NOT     47
Name: category, dtype: int64

DM     615
SYM    308
NOT    196
Name: category, dtype: int64
-----


## Sort the data so that the data is stable

In [21]:
goldstd = goldstd.sort_values(["chemical_id", "disease_id"])

## Save split pieces to file

In [22]:
goldstd.to_csv("data/split_indications/labeled_pharmacotherapydb.tsv", sep = '\t', index = False)

In [23]:
for idx in range(K):
    fname = "holdout"
    holdouts[idx].to_csv("data/{}/{}_piece{}.tsv".format(fname, fname, idx), sep = '\t', index = False)
    
    fname = "training"
    training[idx].to_csv("data/{}/{}_piece{}.tsv".format(fname, fname, idx), sep = '\t', index = False)